# Setup

In [3]:
# import modules
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [4]:
# create spark session
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()

# Data Frames
- Contains row objects
- Can run SQL queries
- Can store with a schema (more efficient)
- Read and write to JSON, Hive, Parquet, etc
- Allows for better interoperability and simplified development

In [31]:
# import data (unstructured) as rdd
friends = spark.sparkContext.textFile('../data/fakefriends.csv')

In [32]:
# create schema
def mapper(line):
    fields = line.split(',')
    return Row(
        id = int(fields[0]), \
        name = str(fields[1].encode('utf-8')), \
        age = int(fields[2]), \
        numFriends = int(fields[3])
    )

# apply schema
friends = friends.map(mapper)

In [33]:
# register dataframe and infer schema
schemaFriends = spark.createDataFrame(friends).cache()
schemaFriends.createOrReplaceTempView('friends')

type(schemaFriends)

pyspark.sql.dataframe.DataFrame

In [34]:
# query dataframe using spark sql
teenagers_df = spark.sql(
    'select * from friends where age >= 13 and age <= 19'
)

# display results
for teen in teenagers_df.collect():
    print(teen)

Row(id=21, name="b'Miles'", age=19, numFriends=268)
Row(id=52, name="b'Beverly'", age=19, numFriends=269)
Row(id=54, name="b'Brunt'", age=19, numFriends=5)
Row(id=106, name="b'Beverly'", age=18, numFriends=499)
Row(id=115, name="b'Dukat'", age=18, numFriends=397)
Row(id=133, name="b'Quark'", age=19, numFriends=265)
Row(id=136, name="b'Will'", age=19, numFriends=335)
Row(id=225, name="b'Elim'", age=19, numFriends=106)
Row(id=304, name="b'Will'", age=19, numFriends=404)
Row(id=341, name="b'Data'", age=18, numFriends=326)
Row(id=366, name="b'Keiko'", age=19, numFriends=119)
Row(id=373, name="b'Quark'", age=19, numFriends=272)
Row(id=377, name="b'Beverly'", age=18, numFriends=418)
Row(id=404, name="b'Kasidy'", age=18, numFriends=24)
Row(id=409, name="b'Nog'", age=19, numFriends=267)
Row(id=439, name="b'Data'", age=18, numFriends=417)
Row(id=444, name="b'Keiko'", age=18, numFriends=472)
Row(id=492, name="b'Dukat'", age=19, numFriends=36)
Row(id=494, name="b'Kasidy'", age=18, numFriends=194)

In [35]:
# query dataframe using pyspark
age_df = schemaFriends \
    .groupBy('age') \
    .count() \
    .orderBy('age')

age_df.show()

+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
| 23|   10|
| 24|    5|
| 25|   11|
| 26|   17|
| 27|    8|
| 28|   10|
| 29|   12|
| 30|   11|
| 31|    8|
| 32|   11|
| 33|   12|
| 34|    6|
| 35|    8|
| 36|   10|
| 37|    9|
+---+-----+
only showing top 20 rows



In [36]:
# stop spark session
spark.stop()